In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
! pwd

The column data are the values for wavelength [nm], CD [mdeg], HT [V], and Abs. The wavelength was collected between 320 - 180 nm at 0.5 nm intervals (281 data points).

In [ ]:
def get_file_names(file_path):
    """
    Extract a list of .csv file names from CD data folder.
    :param file_path: path to .csv file folder.
    :return: list of file names.
    """ 
    file_list = os.listdir(file_path)
    file_names = sorted([file for file in file_list if file.endswith('.csv')])
    
    return file_names

In [ ]:
def get_cd_data(file_path, file):
    """
    Extract data for Wavelength [nm], CD [mdeg], HT [V], and Abs into a Pandas DataFrame, and return 
    Wavelength [nm] and CD [mdeg] values in the range 300 to 180 nm.
    
    This function was written for .csv files generated by the Jasco Spectra Manager software with data recorded on 
    a Jasco J-815 Spectropolarimeter set to scan at the wavelength range 320 to 180 nm at 0.5 nm intervals.
    :param file_path: path to CD file folder
    :param file: file name
    :return: array of x and y values for wavelength and CD
    """
    df = pd.read_csv(file_path+file,
                     skiprows=19, 
                     skipfooter=45, 
                     names=['Wavelength [nm]', 'CD [mdeg]', 'HT [V]', 'Abs'], 
                     engine='python')
    xvalues = df['Wavelength [nm]'].values
    yvalues = df['CD [mdeg]'].values
    
    return xvalues, yvalues

In [ ]:
def get_label_name(file_path, file):
    """
    Extract name of the test substrate after "TITLE," from line one of the file. File name format:
    "TITLE, [protein ratio]_[substrate ratio] [concentration of protein]mM [protein name]_[substrate name].csv"
    e.g. "TITLE, 5_1 10mM protein_substrate.csv".

    This function was written for .csv files generated by the Jasco Spectra Manager software with data recorded on 
    a Jasco J-815 Spectropolarimeter.
    :param file_path: path to CD file folder.
    :param file: file name.
    :return: str: plot label
    """
    data_file = open(file_path+file, 'r')
    title_line = data_file.readline()
    data_file.close
    
    words = title_line.split(',')[1]
    label = words.split(':')[2].strip()
    
    return label

In [ ]:
def plot_cd(file_path, file_names, outfile, default=True):
    """
    Create overlay plot of CD [mdeg] vs Wavelength [nm] between 300 to 180 nm for file in  
    file_names list.
    :param outfile: str: name of output file with extension (.png, .svg, .pdf).
    :param file_path: path to CD file folder.
    :param file_names: list of file names.
    :param default_range: True: y axis limits contrained from -50 to 50 mdeg.
                          False: y axis limits determined by matplotlib.
    :return: None
    """
    # Set figure parameters
    fig, ax = plt.subplots(figsize=(16,9))
    
    # Set x axis coordinates from 180 to 300 nm.
    ax.set_xlim(180, 300)
    ax.set_xticks(np.arange(180, 301, step=10))
    ax.set_xticks(np.arange(180, 301, step=5), minor=True)
    
    # Constrain y axis limits from -50 to 50 mdeg if default is set to True
    if default:
        ax.set_ylim(-50, 50)
        ax.set_yticks(np.arange(-50, 51, step=10))
        ax.set_yticks(np.arange(-50, 51, step=5), minor=True)

    ax.spines['bottom'].set_position('zero') # Move x-axis to intersect at y=0
    ax.spines['right'].set_visible(False) # Eliminate 'right' axes
    ax.spines['top'].set_visible(False) # Eliminate 'top' axes

    ax.set_xlabel('Wavelength (nm)', fontsize=10, loc='right')
    ax.set_ylabel('CD (mdeg)', fontsize=10)

    ax.yaxis.grid()
        
    # Loop over the files in file_names and generate line for each one
    for file in file_names:
        xvalues, yvalues = get_cd_data(file_path, file)
        ax.plot(xvalues, yvalues, label=get_label_name(file_path, file))

    ax.legend()
    
    fig.savefig(outfile, dpi=300, transparent=True)
    
    return

In [ ]:
file_path = input('Enter file path to CD data: ')
outfile = input('Enter name of .png file to save: ')

plot_cd(file_path, get_file_names(file_path), outfile)